In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import joblib
from Transformer import transformer
import bulid_new_data as bnd
from copy import deepcopy
import traing_function as tp
import assortment_process as at
import warnings
import torch
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from alpermodel_1 import conicmc
from collections import Counter

## Generating the new traing data or testing data 

In [ ]:
choice_size = 30 
order_num = 20000
file_name = "train"
#order_num = 10000
#file_name = "test"


#c_rate is \gamma
for c_rate in [5,10,20,50,100,200]:
    next_train_data = bnd.generate_data(order_num,int(choice_size+c_rate))
    next_train_data.to_csv(file_name+"_dataset\\"+str(c_rate)+'_'+str(order_num)+'.csv',index=False)
    
    #transformer get lables for data
    pro = transformer.true_model(next_train_data,order_num) 
    pro = pro.reshape(order_num,int(choice_size+c_rate))
    label = np.zeros(shape = pro.shape)
    for i in range(pro.shape[0]):
        index = np.argmax(np.random.multinomial(1,pro[i]/np.sum(pro[i])))
        label[i,index] = 1
    label = label[:,0:choice_size]
    pro = pro[:,0:choice_size]

    del_feature = ['orderid']
    features = [i for i in next_train_data.columns if i not in del_feature]
    data_numpy = np.array(next_train_data[features]).reshape(order_num,int(choice_size+c_rate),-1)
    np.save(file_name+"_dataset_numpy\\data_"+str(c_rate)+'_'+str(order_num)+'.npy',data_numpy[:,0:choice_size,:])
    np.save(file_name+"_dataset_numpy\\label_"+str(c_rate)+'_'+str(order_num)+'.npy',label)

    #Transform the numpy data
    del_feature = ['orderid']
    features = [i for i in next_train_data.columns if i not in del_feature]
    data_np = np.array(next_train_data[features])
    data_np_cate = np.zeros(shape = data_np.shape)
    for idx,item in enumerate([11, 7, 97, 63, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1]):
        if item !=1 :
            data_np_cate[:,idx] = data_np[:,idx]
            data_np[:,idx] = 1
    
    scaler = MinMaxScaler().fit(data_np)
    data_np = scaler.transform(data_np)
    for idx,item in enumerate([11, 7, 97, 63, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1]):
        if item != 1:
            data_np[:,idx] = 1
    data_np = data_np.reshape(-1,int(choice_size+c_rate),len(features))
    data_np_cate = data_np_cate.reshape(-1,int(choice_size+c_rate),len(features))
    data = np.array([data_np_cate[:,0:choice_size,:],data_np[:,0:choice_size,:]])

    #Incorporate the assortment information for DeemFM-a
    data_temp = np.zeros((order_num, choice_size, 9))
    data_cate_temp = np.zeros((order_num, choice_size, 2))
    for i in range(order_num):
        min1 = np.min(data[1,i,:,5])
        max1 = np.max(data[1,i,:,5])
        mean1 = np.mean(data[1,i,:,5])
        min2 = np.min(data[1,i,:,6])
        max2 = np.max(data[1,i,:,6])
        mean2 = np.mean(data[1,i,:,6])
        min3 = np.min(data[1,i,:,7])
        max3 = np.max(data[1,i,:,7])
        mean3 = np.mean(data[1,i,:,7])
        data_temp[i,:,0] = min1
        data_temp[i,:,1] = max1
        data_temp[i,:,2] = mean1
        data_temp[i,:,3] = min2
        data_temp[i,:,4] = max2
        data_temp[i,:,5] = mean2
        data_temp[i,:,6] = min3
        data_temp[i,:,7] = max3
        data_temp[i,:,8] = mean3 
        for j in range(choice_size):
            if data[1,i,j,6] == np.min(data[1,i,:,6]):
                data_cate_temp[i,j,0] = 1
            if data[1,i,j,6] == np.max(data[1,i,:,6]):
                data_cate_temp[i,j,1] = 1
    data_with_assort_info = np.array([np.concatenate([data[0],np.zeros((order_num, choice_size, 9)),data_cate_temp],axis = 2),np.concatenate([data[1],data_temp,np.ones((order_num,choice_size,2))],axis = 2)])

    np.save(file_name+'_dataset_numpy_normalised\\data_'+str(c_rate)+'_'+str(order_num)+'.npy',data)
    np.save(file_name+'_dataset_numpy_normalised\\label_'+str(c_rate)+'_'+str(order_num)+'.npy',label)
    np.save(file_name+'_dataset_numpy_normalised\\data_with_assort_info_'+str(c_rate)+'_'+str(order_num )+'.npy',data_with_assort_info)

## Generating the decision-test data set

In [ ]:
for c_rate in [200]:
    print(c_rate)
    next_valid_data = bnd.generate_data(1000,100+c_rate)
    del_feature = ['orderid','alternative','orderlabel']
    features = [i for i in next_valid_data.columns if i not in del_feature]
    data_np = np.array(next_valid_data[features])
    scaler = MinMaxScaler().fit(data_np)
    data_np_cate = np.zeros(shape = data_np.shape)
    for idx,item in enumerate([11, 7, 97, 63, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1]):
        if item !=1 :
            data_np_cate[:,idx] = data_np[:,idx]
            data_np[:,idx] = 1
    data_np = scaler.transform(data_np)
    for idx,item in enumerate([11, 7, 97, 63, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1]):
        if item != 1:
            data_np[:,idx] = 1

    data_np = data_np.reshape(-1,100+c_rate,len(features))
    data_np_cate = data_np_cate.reshape(-1,100+c_rate,len(features))
    valid_data = np.array([data_np_cate,data_np])
    next_valid_data.to_csv('new_products\\next_valid_data_'+str(c_rate)+'.csv',index=False)
    np.save('new_products\\valid_data_'+str(c_rate)+'.npy',valid_data)

## 1 MNL-Based Choice Model

### 1.1 Training the MNL model

In [ ]:
PATH = "trained_models_missfeatures\\MNL_parameters_special"+str(c_rate)+'_'+str(order_num)+".pt"
model_MNL = tp.train_data(data,label,NUM_EPOCHS = 100,BATCH_SIZE = 32,LR = 0.01,path = PATH)

In [ ]:
#load the trained model
PATH = "trained_models_missfeatures\\MNL_parameters_special"+str(c_rate)+'_'+str(order_num)+".pt"
model_MNL = torch.load(PATH)

### 1.2 Obtaining the utility of decision-test data set

In [ ]:
X = torch.from_numpy(valid_data[0]).to(torch.long)
weight = torch.from_numpy(valid_data[1]).to(torch.float)
utility_MNL,u0 = model_MNL([X,weight])
utility_MNL = utility_MNL.detach().numpy()
u0 = float(u0[0])

### 1.3 Make assortment decision

In [ ]:
warnings.filterwarnings('ignore') 
cadinality = 30
assortment_MNL = []
for i in tqdm(range(utility_MNL.shape[0])):
    assortment_MNL.append(at.MNL(np.exp(utility_MNL[i,0:100]-u0),price[i,0:100],capacity = cadinality))
products = []
price_MNL = []
for idx,item in enumerate(assortment_MNL):
    for j in range(utility_MNL.shape[1]):
        if j in item:
            products.append(idx*utility_MNL.shape[1]+j)
            price_MNL.append(price[idx,j])
        elif j >= 100:
            products.append(idx*utility_MNL.shape[1]+j)
            price_MNL.append(0)
valid_pro = true_model.true_model(next_valid_data.loc[products],1000)
profit = np.dot(valid_pro,np.array(price_MNL))/1000
print(cadinality,profit)
assortment_array = np.zeros((1000,100))
for i,item in enumerate(assortment_MNL):
    for j in item:
        assortment_array[i,j] = 1
np.save("Assortment_missfearures\\MNL_assortment_special"+str(c_rate)+"_"+str(order_num )+'_'+str(cadinality)+".npy",assortment_array)

## 2 DeepFM-based Choice Model

### 2.1 Training the DeepFM model

In [ ]:
total = data.shape[1]
indices = np.random.permutation(total)
k_fold = 5
training_data = data[:,indices[0:int(total*(k_fold-1)/k_fold)],:,:]
training_label = label[indices[0:int(total*(k_fold-1)/k_fold)],:]
training_valid_data = data[:,indices[int(total*(k_fold-1)/k_fold):],:,:]
training_valid_label = label[indices[int(total*(k_fold-1)/k_fold):],:]
weight = int((training_label.shape[0]*training_label.shape[1]-np.sum(training_label))/np.sum(training_label))

In [ ]:
#load model
PATH = 'trained_models\\DeepFM_parameters_'+str(c_rate)+'_'+str(order_num )+'.pt'
model_DeepFM = tp.train_DeepFM(training_data,training_label,training_valid_data,training_valid_label,weight1 = weight,path = PATH,LR = 0.05)

### 2.2 Getting Probability for decision-test data set

In [ ]:
X = torch.from_numpy(valid_data[0]).to(torch.long)
weight = torch.from_numpy(valid_data[1]).to(torch.float)
utility_DeepFM  = model_DeepFM([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()

### 2.3 Makeing assortment decision

In [ ]:
fcadinality = 30
warnings.filterwarnings('ignore')
assortment_DeepFM = []
for i in range(pro_DeepFM.shape[0]):
    #length = len(assortment_ml[i])
    revenue = pro_DeepFM[i,0:100]*price[i,0:100]
    sort_index = np.argsort(-revenue)
    assortment_DeepFM.append(sort_index[0:cadinality])

products = []
price_DeepFM = []
for idx,item in enumerate(assortment_DeepFM):
    for j in range(valid_data.shape[2]):
        if j in item:
            products.append(idx*valid_data.shape[2]+j)
            price_DeepFM.append(price[idx,j])
        elif j >= 100:
            products.append(idx*valid_data.shape[2]+j)
            price_DeepFM.append(0)
valid_pro = true_model.true_model(next_valid_data.loc[products],1000)
average_profit_ml = np.dot(valid_pro,np.array(price_DeepFM))/1000

## 3 DeepFM-a based choice model

### 3.1 Training the DeepFM-a model

In [ ]:
total = data_with_assort_info.shape[1]
indices = np.random.permutation(total) 
k_fold = 5
training_data = data_with_assort_info[:,indices[0:int(total*(k_fold-1)/k_fold)],:,:]
training_label = label[indices[0:int(total*(k_fold-1)/k_fold)],:]
training_valid_data = data_with_assort_info[:,indices[int(total*(k_fold-1)/k_fold):],:,:]
training_valid_label = label[indices[int(total*(k_fold-1)/k_fold):],:]
weight = int((training_label.shape[0]*training_label.shape[1]-np.sum(training_label))/np.sum(training_label))

In [ ]:
PATH = 'trained_models_missfeatures\\DeepFM_parameters_with_assortment_special'+str(c_rate)+'_'+str(order_num )+'.pt'
model_DeepFM = tp.train_DeepFM(training_data,training_label,training_valid_data,training_valid_label,model_name = 'Assortment',weight1 = weight,LR = 0.05,path = PATH)

In [ ]:
PATH = 'trained_models_missfeatures\\DeepFM_parameters_with_assortment_special'+str(c_rate)+'_'+str(order_num )+'.pt'
model_DeepFM = torch.load(PATH)

### 3.2 Making assortment decision

In [ ]:
cadinality = NULL30
model_DeepFM1 = torch.load('trained_models_missfeatures\\DeepFM_parameters_special'+str(c_rate)+'_'+str(order_num )+'.pt')
X = torch.from_numpy(valid_data[0]).to(torch.long)
weight = torch.from_numpy(valid_data[1]).to(torch.float)
utility_DeepFM  = model_DeepFM1([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()
assortment_DeepFM = []
for i in range(pro_DeepFM.shape[0]):
    #length = len(assortment_ml[i])
    revenue = pro_DeepFM[i,0:100]*price[i,0:100]
    sort_index = np.argsort(-revenue)
    assortment_DeepFM.append(sort_index[0:cadinality])
warnings.filterwarnings('ignore')
assortment_ml = []
for i in tqdm(range(1000)):
    assortment_ml.append(at.ml_assortment_swap(model_DeepFM,valid_data[:,i,0:100,:],price[i,0:100],assortment_DeepFM[i],capacity = cadinality,feature_column = feature_column_swapping))
products = []
price_ml = []
for idx,item in enumerate(assortment_ml):
    for j in range(valid_data.shape[2]):
        if j in item:
            products.append(idx*valid_data.shape[2]+j)
            price_ml.append(price[idx,j])
        elif j >= 100:
            products.append(idx*valid_data.shape[2]+j)
            price_ml.append(0)
valid_pro = true_model.true_model(next_valid_data.loc[products],1000)
average_profit_ml = np.dot(valid_pro,np.array(price_ml))/1000
print(cadinality,average_profit_ml)
assortment_array = np.zeros((1000,100))
for i,item in enumerate(assortment_ml):
    for j in item:
        assortment_array[i,j] = 1   
np.save("Assortment_missfearures\\DeepFM_assortment_swapping_special"+str(c_rate)+"_"+str(order_num )+'_'+str(cadinality)+".npy",assortment_array)